# Preparing the dataset 10 for training (OHE and standardizarion)

## Import libraries and create the needed functions

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
def ohe_and_standized(df):
    # Separate numerical and categorical features
    numerical_features = df.select_dtypes(include=['int', 'float']).columns.tolist()
    categorical_features = df.select_dtypes(include=['object']).columns.tolist()
    
    # Create transformers for preprocessing
    numerical_transformer = Pipeline([('scaler', StandardScaler())])
    categorical_transformer = Pipeline([('onehot', OneHotEncoder(sparse_output=False))])  # Set sparse=False to get dense array
    
    # Combine transformers into a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'  # Preserve the remaining features
    )
        
    # Create a pipeline to apply preprocessing
    pipeline = Pipeline([('preprocessor', preprocessor)])
    transformed_data = pipeline.fit_transform(df)
    
    # Get feature names after preprocessing
    feature_names = []
    feature_names.extend(numerical_features)
    
    # Add one-hot encoded categorical feature names
    if 'cat' in preprocessor.named_transformers_ and categorical_features:
        one_hot_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
        categorical_feature_names = one_hot_encoder.get_feature_names_out(input_features=categorical_features)
        feature_names.extend(categorical_feature_names)
    
    # Add any other remaining columns (e.g., columns you didn't process)
    remaining_cols = [col for col in df.columns if col not in numerical_features + categorical_features]
    feature_names.extend(remaining_cols)
    
    # Convert the transformed data back to a DataFrame for visualization
    transformed_df = pd.DataFrame(transformed_data, columns=feature_names)
    
    # set the indexes as the original dataset
    transformed_df = transformed_df.set_index(df.index)
    
    return transformed_df

In [3]:
df = pd.read_csv('./dataset10.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   YEAR                  231 non-null    int64  
 1   Filename              231 non-null    object 
 2   ORG_TYPE_DESCRIPTION  231 non-null    object 
 3   ORG_CODE              231 non-null    object 
 4   NEW_CODE              231 non-null    object 
 5   ORG_TITLE             231 non-null    object 
 6   SEX_CODE              231 non-null    object 
 7   AGE_BAND_CODE         231 non-null    object 
 8   Rate                  231 non-null    float64
dtypes: float64(1), int64(1), object(7)
memory usage: 16.4+ KB


In [4]:
df.head()

,YEAR,Filename,ORG_TYPE_DESCRIPTION,ORG_CODE,NEW_CODE,ORG_TITLE,SEX_CODE,AGE_BAND_CODE,Rate
0,2020,10A_158CR,NATIONAL,E&W,K04000001,England and Wales,M,'35-64,13.16
1,2020,10A_158CR,NATIONAL,E&W,K04000001,England and Wales,M,'1+,44.86
2,2020,10A_158CR,NATIONAL,E&W,K04000001,England and Wales,M,'65-74,73.80
3,2020,10A_158CR,NATIONAL,E&W,K04000001,England and Wales,M,'05-14,0.09
4,2020,10A_158CR,NATIONAL,E&W,K04000001,England and Wales,M,'75+,432.11


In [5]:
print(df['YEAR'].unique())
print(df['Filename'].unique())

[2020]
['10A_158CR']


### The feature YEAR and Filename have constant values so we dont need them

In [6]:
df = df.drop(["YEAR","Filename"],axis = 1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ORG_TYPE_DESCRIPTION  231 non-null    object 
 1   ORG_CODE              231 non-null    object 
 2   NEW_CODE              231 non-null    object 
 3   ORG_TITLE             231 non-null    object 
 4   SEX_CODE              231 non-null    object 
 5   AGE_BAND_CODE         231 non-null    object 
 6   Rate                  231 non-null    float64
dtypes: float64(1), object(6)
memory usage: 12.8+ KB


In [8]:
df = ohe_and_standized(df)
df.head()

,Rate,ORG_TYPE_DESCRIPTION_NATIONAL,ORG_TYPE_DESCRIPTION_REGIONS,ORG_CODE_A,ORG_CODE_B,ORG_CODE_D,ORG_CODE_E,ORG_CODE_E&W,ORG_CODE_ENG,ORG_CODE_F,...,SEX_CODE_F,SEX_CODE_M,SEX_CODE_P,AGE_BAND_CODE_'01-04,AGE_BAND_CODE_'05-14,AGE_BAND_CODE_'1+,AGE_BAND_CODE_'15-34,AGE_BAND_CODE_'35-64,AGE_BAND_CODE_'65-74,AGE_BAND_CODE_'75+
0,-0.455603,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.252200,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.066507,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.539466,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2.232584,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
df.rename(columns={'Rate': 'Target'}, inplace=True)

## create a CSV file again from the procesed data

In [10]:
df.to_csv('procesed_dataset_10.csv', index=False)